In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from pandas.tseries.offsets import MonthEnd
from datetime import datetime, timedelta, date
import requests
import json 
import ta

In [2]:
#assign dates to dataframe index
FB_df = pd.read_csv("/Users/fahad/project_repo/data/final/FB/FB_normalised.csv", index_col=0)
dates = FB_df.index
FB_df['Date'] =dates

In [6]:
FB_df['Date'] = pd.to_datetime(FB_df.Date, format ="%Y-%m-%d")

In [7]:
FB_2010 = pd.read_csv("/Users/fahad/project_repo/data/final/FB/FB_10yr.csv", index_col=0)


In [10]:
#ADD LABELS TO THE TRAINING AND TEST DATAFRAMES AND RETURN DICTIONARIES WITH DATE:LABEL FORMAT
FB_2010_labels = labelling(FB_2010)


In [11]:
FB_2010_flattened= get_ndays_monthly_flattened(FB_2010,FB_df,30)



In [12]:
FB_2010_labels_df = pd.Series(FB_2010_labels).to_frame("Label")


In [13]:
#CREATE DATAFRAME FROM FLATTENED DICTIONARY OF ARRAYS FOR TRAINING AND TEST DATA
FB_2010_flat_df = pd.DataFrame.from_dict(FB_2010_flattened).T



In [14]:
#JOIN THE LABEL DATAFRAME AND THE FLATTENED DATAFRAME ON THE INDEX WHICH IS THE DATE
FB_2010_flat_df = FB_2010_flat_df.merge(FB_2010_labels_df, left_index=True, right_index=True)



In [15]:
FB_2010_flat_df

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,Label
2019-07-31,0.848635,0.837135,0.822285,0.810833,0.022848,0.000000,0.388731,0.472170,0.481359,0.322074,...,0.898384,0.007633,0.496496,0.245633,0.907954,0.916333,0.544318,0.0,0.0,1
2019-06-28,0.897159,0.900624,0.894418,0.897583,0.010553,0.852261,0.502225,0.472170,0.481359,0.602362,...,0.854684,0.055770,0.416634,0.261216,0.955331,0.964391,0.526335,0.0,0.0,0
2019-05-31,0.869279,0.856900,0.867832,0.859840,0.009241,0.279555,0.163625,0.907954,0.936840,0.512474,...,0.847275,0.012402,0.739486,0.672091,0.961757,0.964391,0.595168,0.0,0.0,0
2019-04-30,0.834872,0.824607,0.830317,0.827352,0.004701,0.706852,0.443190,0.970556,0.964391,0.785926,...,0.806178,0.007153,0.339268,0.112905,0.853591,0.913045,0.578103,0.0,0.0,1
2019-03-29,0.874693,0.884153,0.879931,0.886270,0.024184,0.794134,0.545429,0.882198,0.913045,0.558748,...,0.714523,0.055686,0.230778,0.308748,0.928933,0.884716,0.451905,0.0,0.0,0
2019-02-28,0.741215,0.740953,0.741053,0.739952,0.007980,0.160657,0.191688,0.888947,0.903750,0.450205,...,0.731943,0.012053,0.567900,0.413146,0.851982,0.885212,0.427105,0.0,0.0,0
2019-01-31,0.732765,0.727028,0.726210,0.726237,0.011951,0.412066,0.357238,0.888877,0.884716,0.491339,...,0.656857,0.031984,0.477380,0.321566,0.806793,0.880405,0.585120,0.0,0.0,1
2018-12-31,0.648367,0.662241,0.651408,0.664216,0.068174,0.233545,0.306783,0.813670,0.880405,0.630883,...,0.613007,0.032450,0.656345,0.642430,0.757992,0.776926,0.886614,0.0,0.0,0
2018-11-30,0.593316,0.587222,0.582808,0.578017,0.029444,0.367174,0.276295,0.805740,0.830443,0.611724,...,0.633180,0.028459,0.760351,0.552117,0.655066,0.710431,0.690129,0.0,0.0,1
2018-10-31,0.596251,0.607537,0.600397,0.605448,0.032283,0.541530,0.503375,0.723478,0.742275,0.691979,...,0.686741,0.027766,0.727646,0.487062,0.640218,0.707375,0.707248,0.0,0.0,1


In [16]:
export = FB_2010_flat_df.to_csv("/Users/fahad/project_repo/data/final/FB/FB_2010_pre_signals.csv")

In [8]:
def labelling(dataframe):
    """This generates labels for the dataframes we wish to predict price.
    A label of 1 suggests an increase in price between months while a label of 0 indicates
    a decrease in price. If the current price is greater than the previous price then a"""
    empty_dict = dict()
    prev_close =0
    counter = 0
    for index,cols in dataframe.iterrows():
        counter +=1
        current_close = cols['Close']
        if float(prev_close) <= float(current_close):
            #signals an increase in price between last month and next month
            empty_dict[index] = 1
        elif float(prev_close) >= float(current_close):
            empty_dict[index]=0

        #reassign prev close to the current close     
        prev_close = current_close
    
    return empty_dict

In [9]:
def get_ndays_monthly_flattened(dataframe_period_end,dataframe_subperiod,n):
    """returns a dictionary which contains key value pair of the month-end date as key and
    value is the proceeding n days. Returns it as a dictionary of dataframes.
    dataframe period end refers to the dataframe which will act as the period end i.e monthly end date
    sub_period dataframe refers to the dataframe with the preceeding n days, i.e daily"""
    empty_dict = dict()
    count = 0
    for end in dataframe_period_end.index:
#         print ("month end/:%s" % end)
        
        month_key = dataframe_period_end.index[count+1]
        
#         print ("month key:%s" % month_key)
        
        last_index = dataframe_period_end.index[-1]
        
#         print ("last index:%s " % last_index)
        
        if (month_key != last_index):
            temp_df = dataframe_subperiod.loc[dataframe_subperiod['Date']<end].head(n)
#             print ("\n")
#             print (temp_df.head(5))
            temp_df = temp_df.drop(['Date'], axis=1)
            
            if (len(temp_df.values.flatten())==360):
                empty_dict[month_key]= temp_df.values.flatten()
                count +=1      
            else:
                pass

        else:
            pass
       
    return empty_dict
    